In [ ]:
import torch 
import torch.nn as nn 

In [ ]:
x = torch.randn(4, 1, 420, 540)

size = 40 # patch size
stride = 10 # patch stride
patches = x.unfold(2, size, stride).unfold(3, size, stride)
print(patches.shape)

In [ ]:
a = patches[0,0,10,10,:,:]
print(a.shape)

In [ ]:
import matplotlib.pyplot as plt 
plt.imshow(a)
plt.show()

In [ ]:
fold = nn.Fold(output_size = (4, 1, 420, 540), kernel_size = (40,40))
xx = fold(patches)


In [16]:
from PIL import Image
picture = Image.open('/home/adguerrero/ia_nas/datasets/size_image/img_comfenalco/img_4.png').convert('L')
picture.save('/home/adguerrero/ia_nas/datasets/size_image/img_comfenalco/grayscale/img_4.png')

In [ ]:
from PIL import Image
picture = Image.open('/home/adguerrero/gia_san/datasets/size_image/original/4401-10.png') 

print(picture.size)
file = '4401-10.jpg'
picture.save("/home/adguerrero/gia_san/datasets/size_image/jpg_compression/Compressed_"+ file,  
                "JPEG", 
                optimize = True,  
                quality = 70) 

In [ ]:
import os
os.path.getsize('/home/adguerrero/Documents/mejoramiento_imagenes/Compressed_29386302_3603_00001.jpg')

In [ ]:
import os
os.path.getsize('/home/adguerrero/gia_san/datasets/snr_escrituras/29386302/29386302_3603_00001.png')

In [ ]:
from PIL import Image
path = '/home/adguerrero/gia_san/datasets/size_image/'

h,w = picture.size 
# thresh = 200
picture = Image.open(path + 'original/4401-10.png').convert('L')
reduccion = picture.resize((int(h*0.5),int(w*0.5)), Image.ANTIALIAS)

# fn = lambda x : 255 if x > thresh else 0
# r = reduccion.convert('L').point(fn, mode='1')
reduccion.save(path + 'antialiasing/4401-10.png',optimize=True) 

In [67]:
from PIL import Image
path = '/home/adguerrero/gia_san/datasets/size_image/'

h,w = picture.size 
picture = Image.open(path + 'original/4401-10.png')
reduccion = picture.resize((int(h*0.5),int(w*0.5)), Image.ANTIALIAS)

file = '4401-10.jpg'
reduccion.save(path + "antialias_jpgcompre/Compressed_"+ file,  
                "JPEG", 
                optimize = True,  
                quality = 70) 


In [1]:
import cv2 as cv
import numpy as np

def image_process(image: np.ndarray) -> np.ndarray:

    raw_img = image.copy()
    rows, cols = image.shape
    blur_ksize = (4, 4) if cols > 2000 else (2, 2)

    # step 1: reduce background noise
    morph_size = 1
    image = cv.blur(image, ksize=blur_ksize)
    image = cv.threshold(image, thresh=140, maxval=255, type=cv.THRESH_BINARY)[1]

    kernel = cv.getStructuringElement(
        shape=cv.MORPH_RECT,
        ksize=(2*morph_size + 1, 2*morph_size + 1),
        anchor=(morph_size, morph_size)
        )
    ## to black background and white foreground
    image = cv.bitwise_not(image)
    image = cv.morphologyEx(image, op=cv.MORPH_OPEN, kernel=kernel, iterations=1)

    # step 2: remove vertical and horizontal lines
    dil_mask_ksize = (6, 2) if cols > 2000 else (3, 2)
    kernel = cv.getStructuringElement(shape=cv.MORPH_RECT, ksize=dil_mask_ksize)
    dilate_mask = cv.morphologyEx(image.copy(), op=cv.MORPH_DILATE, kernel=kernel)

    vert_kernel = cv.getStructuringElement(shape=cv.MORPH_RECT, ksize=(1, int(rows * 0.03)))
    hori_kernel = cv.getStructuringElement(shape=cv.MORPH_RECT, ksize=(int(cols * 0.03), 1))

    vert_mask = cv.morphologyEx(dilate_mask.copy(), op=cv.MORPH_OPEN, kernel=vert_kernel, iterations=2)
    vert_mask = cv.dilate(
        vert_mask,
        kernel=cv.getStructuringElement(cv.MORPH_RECT, (3, 1)),
        iterations=2
        )

    hori_mask = cv.morphologyEx(dilate_mask.copy(), op=cv.MORPH_OPEN, kernel=hori_kernel, iterations=2)
    hori_mask = cv.dilate(
        hori_mask,
        kernel=cv.getStructuringElement(cv.MORPH_RECT, (1, 3)),
        iterations=2
        )

    lines_mask = cv.bitwise_or(vert_mask, hori_mask)
    ## to white background and black foreground
    final_mask = cv.bitwise_not(image)
    ## apply mask to remove lines
    final_mask[lines_mask == 255] = 255

    # step 3: recover original characters
    # to black background again
    final_mask = cv.bitwise_not(final_mask)
    final_mask = cv.dilate(
        final_mask,
        kernel=cv.getStructuringElement(cv.MORPH_RECT, (2, 2)), anchor=(0, 0),
        iterations=2
        )

    result_img = cv.bitwise_and(raw_img, final_mask)
    result_img[final_mask == 0] = 255

    return result_img

In [7]:
img = cv.imread('/home/adguerrero/ia_nas/datasets/autoencoder/dataset/result_ocr/prueba/original_noisy.png',cv.IMREAD_GRAYSCALE)
result_img = image_process(img)
cv.imwrite('./preprocess.png',result_img)

True

In [21]:
from PIL import Image
from itertools import product

def tile(filename, dir_in, dir_out, type, d):
    name, ext = os.path.splitext(filename)
    img_N = Image.open(os.path.join(dir_in, type, filename))
    img_C = Image.open(os.path.join(dir_in, type, filename))
    w, h = img_N.size
    
    i3 = 0 
    i4 = 0 
    i5 = 0
    grid = product(range(0, h-h%d, d), range(0, w-w%d, d))
    for i, j in grid:
        box = (j, i, j+d, i+d)
        out_N = os.path.join(dir_out, type, f'{name}_{i3}{i4}{i5}{ext}')
        img_N.crop(box).save(out_N)

        i5 = i5 + 1
        if i5 > 9:
            i5 = 0
            i4 = i4 +1 
            if i4 > 9:
                i4 = 0
                i3 = i3 + 1

In [22]:
tile('4401-1.png','./', './crop/', 'noisy', 128)